[View in Colaboratory](https://colab.research.google.com/github/FatmaSayedAhmed/PhDResearch/blob/master/DR1_GPU_2Classes_InceptionV3_FineTuning_PreTrainedNetwork_ipynb.ipynb)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


### Class 0 Vs Class 1

In [0]:
import tensorflow as tf 
import keras
import os

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model

os.environ["CUDA_VISIBLE_DEVICES"] = '0' #use GPU with ID=0
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7 # maximun alloc gpu50% of MEM
#config.gpu_options.allow_growth = True #allocate dynamically
sess = tf.Session(config = config)
keras.backend.set_session(sess)

# path to the model weights files.
#weights_path = '../vgg16_weights.h5'

top_model_weights_path = '/content/gdrive/My Drive/Colab Notebooks/bottleneck_fc_model_InceptionV3_PTN_data1.h5'

# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = '/content/gdrive/My Drive/Colab Notebooks/data1_Train1708_Test732/train'
validation_data_dir = '/content/gdrive/My Drive/Colab Notebooks/data1_Train1708_Test732/validation'
nb_train_samples = 3416
nb_validation_samples = 1464

epochs = 50
batch_size = 8

# build the VGG16 network
base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width,img_height,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
# model.add(top_model)
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))



# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model
model.fit_generator(
    train_generator,
    #samples_per_epoch=nb_train_samples,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
#nb_val_samples=nb_validation_samples
validation_steps = nb_validation_samples)


print('-------------------------------------------------------------------')
print('------------------ Evalute the train set ------------------------')
print('-------------------------------------------------------------------')

train_score = model.evaluate_generator(generator=train_generator)
print('Train loss:', train_score[0])
print('Train accuracy:', train_score[1])

print('-------------------------------------------------------------------')
print('------------------ Evalute the validation set ------------------------')
print('-------------------------------------------------------------------')

validation_score = model.evaluate_generator(generator=validation_generator)
print('validation loss:', validation_score[0])
print('validation accuracy:', validation_score[1])


print('-------------------------------------------------------------------')
print('------------------ Train Done -------------------------------------')
print('-------------------------------------------------------------------')